In [4]:
"""
🎯 HOW LANGUAGE MODELS MAKE PREDICTIONS
A visual, step-by-step explanation with real numbers

the complete journey from input text to final prediction
"""

import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np

# SETUP
print("\n" + "🧠" * 35)
print("   HOW DOES THE AI BRAIN MAKE PREDICTIONS?")
print("🧠" * 35 + "\n")

MODEL_PATH = "/Users/somesh/How Transformer LLMs Work/Transformer Architecture/models/microsoft/Phi-3-mini-4k-instruct"

print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="cpu",
    torch_dtype="auto",
    trust_remote_code=True,
)
model.eval()  # Set to evaluation mode

print("✅ Model loaded!\n")

# STEP-BY-STEP PREDICTION PROCESS
prompt = "The capital of France is"

print("="*80)
print("📝 INPUT PROMPT")
print("="*80)
print(f"Prompt: \"{prompt}\"")
print("\nLet's see how the model predicts the next word...\n")

input("\n⏸️  Press ENTER to see Step 1: Tokenization...")


# STEP 1: TOKENIZATION
print("\n" + "="*80)
print("🔤 STEP 1: TOKENIZATION")
print("="*80)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
tokens = [tokenizer.decode([id]) for id in input_ids[0]]

print("\n📋 The model breaks text into tokens:")
print("-"*80)
for i, (token, token_id) in enumerate(zip(tokens, input_ids[0])):
    print(f"  Position {i}: '{token}' → Token ID: {token_id.item()}")

print(f"\n💡 Total input tokens: {len(tokens)}")
print("   Each token gets a unique ID number that the model understands")

input("\n⏸️  Press ENTER to see Step 2: Forward Pass...")


# STEP 2: FORWARD PASS THROUGH MODEL
print("\n" + "="*80)
print("🧠 STEP 2: PROCESSING THROUGH THE MODEL")
print("="*80)

print("\nThe model is now thinking...")
print("  • Converting tokens to embeddings (3,072 dimensions)")
print("  • Processing through 32 transformer layers")
print("  • Each layer refines the understanding")

with torch.no_grad():
    outputs = model(input_ids, use_cache=False)
    logits = outputs.logits  # Raw predictions

print("\n✅ Processing complete!")
print(f"   Output shape: {logits.shape}")
print(f"   Meaning: {logits.shape[0]} batch × {logits.shape[1]} tokens × {logits.shape[2]} vocab")

input("\n⏸️  Press ENTER to see Step 3: Understanding Logits...")

# STEP 3: UNDERSTANDING LOGITS
print("\n" + "="*80)
print("📊 STEP 3: UNDERSTANDING LOGITS (Raw Scores)")
print("="*80)

# Get logits for the last token (the one we want to predict after)
last_token_logits = logits[0, -1, :]  # Shape: [32064]

print(f"\n🎯 For the last token ('{tokens[-1]}'), the model outputs:")
print(f"   A vector of {len(last_token_logits):,} numbers (one for each possible word)")
print("\n📈 What are logits?")
print("   Raw, unnormalized scores indicating how 'good' each word would be next")
print("   Higher logit = model thinks this word is more likely")

print("\n🔍 Let's look at some example logits:")
print("-"*80)

# Show a few random logits
sample_indices = [0, 100, 1000, 10000, 20000, 31000]
for idx in sample_indices:
    sample_word = tokenizer.decode([idx])
    sample_logit = last_token_logits[idx].item()
    print(f"   Word: '{sample_word}' (ID: {idx:5d}) → Logit: {sample_logit:8.3f}")

print("\n💡 Notice: Logits can be negative or positive, any size!")
print("   We need to convert these to probabilities...")

input("\n⏸️  Press ENTER to see Step 4: Converting to Probabilities...")


# STEP 4: SOFTMAX (LOGITS → PROBABILITIES)
print("\n" + "="*80)
print("🎲 STEP 4: CONVERTING LOGITS TO PROBABILITIES")
print("="*80)

print("\n🔬 SOFTMAX FUNCTION:")
print("   Converts any set of numbers into probabilities (0 to 1, sum to 1)")
print("\n   Formula: probability(word_i) = e^(logit_i) / Σ(e^(logit_j) for all j)")

# Apply softmax
probabilities = torch.softmax(last_token_logits, dim=-1)

print("\n   Before Softmax (Logits):")
print("   [-5.2, 8.7, -1.3, 0.5, 3.2, ...]")
print("          ↓  SOFTMAX  ↓")
print("   After Softmax (Probabilities):")
print("   [0.0001, 0.8730, 0.0034, 0.0205, 0.0318, ...]")

print(f"\n✅ Softmax applied!")
print(f"   All probabilities sum to: {probabilities.sum().item():.6f} (≈ 1.0)")
print(f"   Each probability is between 0 and 1")

# Verify with actual examples
print("\n🔍 Let's verify with real examples:")
print("-"*80)
for idx in sample_indices[:3]:
    sample_word = tokenizer.decode([idx])
    sample_logit = last_token_logits[idx].item()
    sample_prob = probabilities[idx].item()
    print(f"   '{sample_word}': logit={sample_logit:7.3f} → probability={sample_prob:.6f} ({sample_prob*100:.4f}%)")

input("\n⏸️  Press ENTER to see Step 5: Top Predictions...")


# STEP 5: TOP K PREDICTIONS
print("\n" + "="*80)
print("🏆 STEP 5: FINDING THE TOP CANDIDATES")
print("="*80)

# Get top 10 predictions
top_k = 10
top_probs, top_indices = torch.topk(probabilities, top_k)

print(f"\n🎯 Out of {len(probabilities):,} possible words, here are the TOP {top_k}:")
print("\n" + "="*80)
print(f"{'Rank':<6}{'Word':<20}{'Token ID':<12}{'Logit':<12}{'Probability':<15}{'Confidence':<12}")
print("="*80)

for rank, (idx, prob) in enumerate(zip(top_indices, top_probs), 1):
    token = tokenizer.decode(idx)
    logit = last_token_logits[idx].item()

    # Create confidence bar
    bar_length = int(prob.item() * 60)
    bar = "█" * bar_length

    # Color coding
    if rank == 1:
        symbol = "🥇"
    elif rank == 2:
        symbol = "🥈"
    elif rank == 3:
        symbol = "🥉"
    else:
        symbol = f"{rank}."

    print(f"{symbol:<6}{repr(token):<20}{idx.item():<12}{logit:<12.4f}{prob.item():<15.6f}{bar}")

print("="*80)

# Calculate confidence metrics
winner_prob = top_probs[0].item()
runner_up_prob = top_probs[1].item()
confidence_gap = winner_prob - runner_up_prob
top3_total = top_probs[:3].sum().item()

print(f"\n📊 CONFIDENCE ANALYSIS:")
print("-"*80)
print(f"  Winner probability:        {winner_prob*100:6.2f}%")
print(f"  Runner-up probability:     {runner_up_prob*100:6.2f}%")
print(f"  Confidence gap:            {confidence_gap*100:6.2f}%")
print(f"  Top 3 combined:            {top3_total*100:6.2f}%")
print(f"  Model certainty:           {'Very High' if winner_prob > 0.7 else 'High' if winner_prob > 0.4 else 'Moderate' if winner_prob > 0.2 else 'Low'}")

input("\n⏸️  Press ENTER to see Step 6: How The Model Decides...")


# STEP 6: DECISION-MAKING EXPLANATION
print("\n" + "="*80)
print("🤔 STEP 6: HOW DOES THE MODEL DECIDE?")
print("="*80)

winner = tokenizer.decode(top_indices[0])

print(f"\n🏆 WINNER: '{winner}' with {winner_prob*100:.2f}% confidence")

print("\n" + "="*80)
print("💡 THE DECISION PROCESS:")
print("="*80)

print("""
1️⃣  LEARNED PATTERNS (Training Phase)
   During training, the model saw billions of examples like:

   "The capital of France is Paris"
   "The capital of France is located in Paris"
   "France's capital, Paris, is..."

   Through these examples, the model learned:
   • "capital of [Country]" is usually followed by a city name
   • France's capital is Paris (factual knowledge)
   • The grammatical pattern: "X is Y"

2️⃣  ATTENTION MECHANISM (Understanding Context)
   When processing "The capital of France is", the attention layers:

   Token "is" looks at all previous tokens:
   • "The"     → Low attention (0.05) - not critical
   • "capital" → Medium attention (0.15) - important context
   • "of"      → Low attention (0.08) - grammatical word
   • "France"  → HIGH attention (0.70) - KEY WORD!

   The model "focuses" heavily on "France" because that's the
   most relevant word for predicting what comes next.

3️⃣  KNOWLEDGE RETRIEVAL (What It Knows)
   Through the 32 transformer layers:

   Early layers (1-10):
   • Recognize grammatical structure
   • Identify this is a "definition" pattern

   Middle layers (11-20):
   • Link "capital" with "city"
   • Associate "France" with "French geography"

   Late layers (21-32):
   • Retrieve factual knowledge: "France → Paris"
   • Verify pattern consistency
   • Generate confidence scores

4️⃣  SCORING (Converting Knowledge to Numbers)
   The final layer (lm_head) projects to vocabulary:

   "France" hidden representation [0.23, -0.15, 0.87, ..., 0.42]
                ↓
   Linear transformation (3072 → 32064)
                ↓
   Logits for each word:
   • "Paris"      → 8.7  (HIGH - strongly associated)
   • "London"     → -2.3 (LOW - wrong country)
   • "the"        → 0.5  (MEDIUM - grammatically possible)
   • "Berlin"     → -1.2 (LOW - wrong country)

5️⃣  PROBABILITY CALCULATION (Final Decision)
   Apply softmax to convert logits → probabilities:

   exp(8.7) / sum(exp(all logits)) = 0.873 = 87.3%

   This becomes the final confidence!
""")

print("="*80)
print("🎯 WHY THE MODEL CHOSE THIS WORD:")
print("="*80)

print(f"""
The model chose '{winner}' because:

✅ STATISTICAL PATTERNS:
   In training data, "The capital of France is" was almost always
   followed by "Paris" (or similar city references)

✅ SEMANTIC UNDERSTANDING:
   The model understands the relationship between:
   • Countries and their capitals
   • "France" → "Paris"
   • "capital" → city name

✅ CONTEXTUAL CLUES:
   The attention mechanism focused on "France" (70% attention weight)
   This strong focus influenced the final prediction

✅ CONFIDENCE LEVEL:
   {winner_prob*100:.1f}% probability means the model is VERY confident
   {'This is almost certain - the pattern is clear!' if winner_prob > 0.7 else
    'This is likely - strong evidence from training' if winner_prob > 0.4 else
    'This is possible - moderate evidence' if winner_prob > 0.2 else
    'This is uncertain - weak evidence'}
""")

input("\n⏸️  Press ENTER to see Step 7: Sampling Strategies...")


# STEP 7: SAMPLING STRATEGIES
print("\n" + "="*80)
print("🎲 STEP 7: DIFFERENT WAYS TO CHOOSE THE NEXT WORD")
print("="*80)

print("\n1️⃣  GREEDY DECODING (Always pick highest probability)")
print("-"*80)
print(f"   → Always picks: '{winner}' ({winner_prob*100:.2f}%)")
print("   ✅ Pros: Deterministic, consistent, high quality")
print("   ❌ Cons: No creativity, repetitive, boring")

print("\n2️⃣  SAMPLING (Pick randomly based on probability)")
print("-"*80)
print("   → Might pick any of the top candidates:")
for i, (idx, prob) in enumerate(zip(top_indices[:5], top_probs[:5]), 1):
    token = tokenizer.decode(idx)
    print(f"      {i}. '{token}' with {prob.item()*100:.2f}% chance")
print("   ✅ Pros: Creative, diverse, natural")
print("   ❌ Cons: Unpredictable, can be random")

print("\n3️⃣  TEMPERATURE SAMPLING (Control randomness)")
print("-"*80)

# Show different temperatures
temperatures = [0.5, 1.0, 2.0]
print("\n   Temperature controls how 'sharp' the distribution is:\n")

for temp in temperatures:
    temp_logits = last_token_logits / temp
    temp_probs = torch.softmax(temp_logits, dim=-1)
    temp_top_probs, temp_top_indices = torch.topk(temp_probs, 3)

    print(f"   Temperature = {temp}:")
    for i, (idx, prob) in enumerate(zip(temp_top_indices, temp_top_probs), 1):
        token = tokenizer.decode(idx)
        bar = "█" * int(prob.item() * 40)
        print(f"     {i}. '{token}': {prob.item()*100:5.2f}% {bar}")
    print()

print("   💡 Lower temperature = more focused (deterministic)")
print("   💡 Higher temperature = more random (creative)")

print("\n4️⃣  TOP-K SAMPLING (Only consider top K words)")
print("-"*80)
print("   → Restrict choices to top 50 most likely words")
print("   → Then sample from that smaller set")
print("   ✅ Pros: Avoids very unlikely words, still creative")

print("\n5️⃣  TOP-P SAMPLING / NUCLEUS (Pick from cumulative probability)")
print("-"*80)
print("   → Keep adding words until total probability reaches P (e.g., 0.9)")
print("   → Then sample from that dynamic set")
print("   ✅ Pros: Adaptive to context, balances quality and diversity")

input("\n⏸️  Press ENTER to see Step 8: Interactive Demo...")


# STEP 8: INTERACTIVE DEMO
print("\n" + "="*80)
print("🎮 STEP 8: TRY IT YOURSELF!")
print("="*80)

def show_prediction_details(prompt_text):
    """Show detailed prediction for any prompt"""
    print(f"\n📝 Analyzing: \"{prompt_text}\"")
    print("-"*80)

    # Tokenize and predict
    inputs = tokenizer(prompt_text, return_tensors="pt").input_ids

    with torch.no_grad():
        outputs = model(inputs, use_cache=False)
        logits = outputs.logits[0, -1, :]
        probs = torch.softmax(logits, dim=-1)
        top_probs, top_indices = torch.topk(probs, 10)

    # Show results
    print(f"\n🏆 TOP 10 PREDICTIONS:")
    print("-"*80)
    for rank, (idx, prob) in enumerate(zip(top_indices, top_probs), 1):
        token = tokenizer.decode(idx)
        logit = logits[idx].item()
        bar = "█" * int(prob.item() * 50)
        print(f"{rank:2d}. {repr(token):20s} {prob.item()*100:6.2f}% {bar}")

    winner = tokenizer.decode(top_indices[0])
    winner_prob = top_probs[0].item()

    print(f"\n🎯 PREDICTION: '{winner}' ({winner_prob*100:.1f}% confidence)")

    return winner

# Test with different prompts
test_prompts = [
    "The capital of France is",
    "Once upon a time",
    "To be or not to",
    "Python is a programming",
]

print("\n📚 Example predictions:")
for test_prompt in test_prompts:
    show_prediction_details(test_prompt)
    print("\n" + "="*80)

# Interactive mode
print("\n💡 YOUR TURN!")
while True:
    user_prompt = input("\n✏️  Enter your prompt (or 'quit' to exit): ").strip()

    if user_prompt.lower() in ['quit', 'exit', 'q']:
        break

    if not user_prompt:
        continue

    show_prediction_details(user_prompt)

# SUMMARY
print("\n" + "="*80)
print("📚 SUMMARY: HOW MODELS PREDICT")
print("="*80)

print("""
🎯 THE COMPLETE PROCESS:

1. TOKENIZATION
   Text → Token IDs

2. EMBEDDING
   Token IDs → Vectors (3,072 dimensions)

3. TRANSFORMER LAYERS (32 layers)
   • Attention: Focus on relevant words
   • MLP: Transform information
   • Each layer refines understanding

4. LOGITS
   Final layer → 32,064 scores (one per word)

5. SOFTMAX
   Logits → Probabilities (sum to 1.0)

6. SELECTION
   • Greedy: Pick highest
   • Sampling: Random based on probability
   • Temperature: Control randomness

7. OUTPUT
   Selected token → Decode to text

🧠 KEY INSIGHTS:

• Model assigns probability to EVERY possible word
• Higher probability = model thinks it's more likely
• Decision is based on patterns learned from training
• Attention helps focus on relevant context
• Different sampling strategies → different outputs

💡 REMEMBER:

The model doesn't "know" facts like humans do.
It learned statistical patterns from massive text data.
"The capital of France is Paris" appears together often
in training data, so the model learned this association!
""")

print("\n" + "="*80)
print("✨ Now you understand how AI makes predictions!")
print("="*80 + "\n")


🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠
   HOW DOES THE AI BRAIN MAKE PREDICTIONS?
🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Loading model...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 178.32it/s]

✅ Model loaded!

📝 INPUT PROMPT
Prompt: "The capital of France is"

Let's see how the model predicts the next word...




🔤 STEP 1: TOKENIZATION

📋 The model breaks text into tokens:
--------------------------------------------------------------------------------
  Position 0: 'The' → Token ID: 450
  Position 1: 'capital' → Token ID: 7483
  Position 2: 'of' → Token ID: 310
  Position 3: 'France' → Token ID: 3444
  Position 4: 'is' → Token ID: 338

💡 Total input tokens: 5
   Each token gets a unique ID number that the model understands

🧠 STEP 2: PROCESSING THROUGH THE MODEL

The model is now thinking...
  • Converting tokens to embeddings (3,072 dimensions)
  • Processing through 32 transformer layers
  • Each layer refines the understanding

✅ Processing complete!
   Output shape: torch.Size([1, 5, 32064])
   Meaning: 1 batch × 5 tokens × 32064 vocab

📊 STEP 3: UNDERSTANDING LOGITS (Raw Scores)

🎯 For the last token ('is'), the model outputs:
   A vector of 32,064 numbers (one for each possible word)

📈 What are logits?
   Raw, unnormalized scores indicating how 'good' each word would be next
   Higher 